### [Pytorch 기반 회귀 모델 구현]
- Layer : Full-Connected Layer (linear)
- 손실함수 : MSELoss, MAELoss, ...

In [1]:
import pandas as pd

filename = '../../data/BostonHousing.csv'

bostonDF = pd.read_csv(filename, encoding='utf-8')

featureDF = bostonDF[bostonDF.columns[:-1]]
targetSR = bostonDF[bostonDF.columns[-1]]

In [2]:
from sklearn.model_selection import train_test_split

### 훈련용, 테스트용 데이터 분할
x_train, x_test, y_train, y_test = train_test_split(featureDF, targetSR, test_size=0.1, random_state=42)

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(455, 13) (455,) (51, 13) (51,)


In [3]:
import torch
from torch.utils.data import TensorDataset

### pytorch Dataset 및 DataLoader 생성
trainDS = TensorDataset(torch.tensor(x_train.values, dtype=torch.float32),
                        torch.tensor(y_train.values, dtype=torch.float32))
testDS = TensorDataset(torch.tensor(x_test.values, dtype=torch.float32),
                       torch.tensor(y_test.values, dtype=torch.float32))

In [4]:
from torch.utils.data import DataLoader

n_features = featureDF.shape[1]
batch_size = 32
trainDL = DataLoader(trainDS, batch_size=batch_size, shuffle=True)
testDL = DataLoader(testDS, batch_size=batch_size, shuffle=False)

In [5]:
import torch.nn as nn

# 모델 정의
class BostonHousingRegressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(n_features, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

In [6]:
import torch.optim as optim

# 모델, 손실 함수, 옵티마이저 생성
model = BostonHousingRegressor()

# model = nn.Sequential(
#     nn.Linear(13, 100),
#     nn.ReLU(),
#     nn.Linear(100, 50),
#     nn.ReLU(),
#     nn.Linear(50, 1)
# )

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [7]:
for feature, target in trainDL:
    print(feature.shape, target.shape)

torch.Size([32, 13]) torch.Size([32])
torch.Size([32, 13]) torch.Size([32])
torch.Size([32, 13]) torch.Size([32])
torch.Size([32, 13]) torch.Size([32])
torch.Size([32, 13]) torch.Size([32])
torch.Size([32, 13]) torch.Size([32])
torch.Size([32, 13]) torch.Size([32])
torch.Size([32, 13]) torch.Size([32])
torch.Size([32, 13]) torch.Size([32])
torch.Size([32, 13]) torch.Size([32])
torch.Size([32, 13]) torch.Size([32])
torch.Size([32, 13]) torch.Size([32])
torch.Size([32, 13]) torch.Size([32])
torch.Size([32, 13]) torch.Size([32])
torch.Size([7, 13]) torch.Size([7])


In [10]:
# 학습
num_epochs = 1000
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (inputs, targets) in enumerate(trainDL):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets.unsqueeze(1))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(trainDL):.4f}')

Epoch [10/1000], Loss: 51.9879
Epoch [20/1000], Loss: 41.8255
Epoch [30/1000], Loss: 33.8365
Epoch [40/1000], Loss: 32.1901
Epoch [50/1000], Loss: 26.3588
Epoch [60/1000], Loss: 24.6051
Epoch [70/1000], Loss: 21.5961
Epoch [80/1000], Loss: 19.8463
Epoch [90/1000], Loss: 26.3711
Epoch [100/1000], Loss: 18.1051
Epoch [110/1000], Loss: 21.4572
Epoch [120/1000], Loss: 19.0655
Epoch [130/1000], Loss: 18.3485
Epoch [140/1000], Loss: 15.9760
Epoch [150/1000], Loss: 15.8149
Epoch [160/1000], Loss: 16.8325
Epoch [170/1000], Loss: 19.6863
Epoch [180/1000], Loss: 14.4887
Epoch [190/1000], Loss: 14.2781
Epoch [200/1000], Loss: 14.2443
Epoch [210/1000], Loss: 14.5268
Epoch [220/1000], Loss: 15.8389
Epoch [230/1000], Loss: 13.9681
Epoch [240/1000], Loss: 14.9325
Epoch [250/1000], Loss: 15.8545
Epoch [260/1000], Loss: 12.7605
Epoch [270/1000], Loss: 13.9374
Epoch [280/1000], Loss: 12.7172
Epoch [290/1000], Loss: 14.6133
Epoch [300/1000], Loss: 14.3789
Epoch [310/1000], Loss: 14.5132
Epoch [320/1000],

In [11]:
# 테스트 세트 평가
model.eval()
with torch.no_grad():
    test_loss = 0.0
    for inputs, targets in testDL:
        outputs = model(inputs)
        loss = criterion(outputs, targets.unsqueeze(1))
        test_loss += loss.item()
    print(f'Test Loss: {test_loss/len(testDL):.4f}')

Test Loss: 15.3693
